In [218]:
import pandas as pd
import numpy as np
import recordlinkage as rl
import re

In [219]:
pokemon = pd.read_json('Dataset integrati/pokemon_integrati.json.gz', compression='gzip', orient='records', lines=True)
pokemon.head()

,index,#,Name,Generation,Rarity,Evolves_from,Has_gender_diff,Type1,Type2,Total,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Variant,Varieties,sprite_url
0,0,1,Bulbasaur,1,Common,None,False,Grass,Poison,318,45,49,49,65,65,45,None,"[{'is_default': True, 'pokemon': {'name': 'bul...",https://img.pokemondb.net/sprites/sword-shield...
1,1,2,Ivysaur,1,Common,Bulbasaur,False,Grass,Poison,405,60,62,63,80,80,60,None,"[{'is_default': True, 'pokemon': {'name': 'ivy...",https://img.pokemondb.net/sprites/sword-shield...
2,2,3,Venusaur,1,Common,Ivysaur,True,Grass,Poison,525,80,82,83,100,100,80,None,"[{'is_default': True, 'pokemon': {'name': 'ven...",https://img.pokemondb.net/sprites/sword-shield...
3,3,3,Venusaur,1,Common,Ivysaur,True,Grass,Poison,625,80,100,123,122,120,80,Mega Venusaur,"[{'is_default': True, 'pokemon': {'name': 'ven...",https://img.pokemondb.net/sprites/sword-shield...
4,4,4,Charmander,1,Common,None,False,Fire,None,309,39,52,43,60,50,65,None,"[{'is_default': True, 'pokemon': {'name': 'cha...",https://img.pokemondb.net/sprites/sword-shield...


In [220]:
moves = pd.read_json('Dataset integrati/moves_integrati.json.gz', compression='gzip', orient='records', lines=True)
moves.head()

,index,Name,Introducted_in,Type_x,Type_y,Power_x,Power_y,Accuracy,Acc.,PP_x,PP_y,Damage_class,Effect,Prob. (%),Learned_by
0,0,Absorb,1,Grass,Grass,20.0,20,100.0,100,25,25,Special,User recovers half the HP inflicted on opponent.,—,"[{'name': 'zubat', 'url': 'https://pokeapi.co/..."
1,1,Accelerock,7,Rock,Rock,40.0,40,100.0,100,20,20,Physical,User attacks first.,—,"[{'name': 'lycanroc-midday', 'url': 'https://p..."
2,2,Acid,1,Poison,Poison,40.0,40,100.0,100,30,30,Special,May lower opponent's Special Defense.,10,"[{'name': 'ekans', 'url': 'https://pokeapi.co/..."
3,3,Acid Armor,1,Poison,Poison,NaN,—,NaN,—,20,20,Status,Sharply raises user's Defense.,—,"[{'name': 'tentacool', 'url': 'https://pokeapi..."
4,4,Acid Spray,5,Poison,Poison,40.0,40,100.0,100,20,20,Special,Sharply lowers opponent's Special Defense.,100,"[{'name': 'ekans', 'url': 'https://pokeapi.co/..."


In [221]:
abilities = pd.read_json('Dataset integrati/Abilities_integrati.json.gz', compression='gzip', orient='records', lines=True)
abilities.head()

,index,Name,Generation,Description,Pokemon
0,0,Adaptability,4,Powers up moves of the same type.,"[{'is_hidden': False, 'pokemon': {'name': 'eev..."
1,1,Aerilate,6,Turns Normal-type moves into Flying-type moves.,"[{'is_hidden': False, 'pokemon': {'name': 'pin..."
2,2,Aftermath,4,Damages the attacker landing the finishing hit.,"[{'is_hidden': True, 'pokemon': {'name': 'volt..."
3,3,Air Lock,3,Eliminates the effects of weather.,"[{'is_hidden': False, 'pokemon': {'name': 'ray..."
4,4,Analytic,5,Boosts move power when the Pokémon moves last.,"[{'is_hidden': True, 'pokemon': {'name': 'magn..."


# PONTE ABILITA' - POKEMON

In [222]:
abil_poke = abilities[["Name", "Pokemon"]]
abil_poke.head()

,Name,Pokemon
0,Adaptability,"[{'is_hidden': False, 'pokemon': {'name': 'eev..."
1,Aerilate,"[{'is_hidden': False, 'pokemon': {'name': 'pin..."
2,Aftermath,"[{'is_hidden': True, 'pokemon': {'name': 'volt..."
3,Air Lock,"[{'is_hidden': False, 'pokemon': {'name': 'ray..."
4,Analytic,"[{'is_hidden': True, 'pokemon': {'name': 'magn..."


In [223]:
for index, row in abil_poke.iterrows():
    row['Pokemon'] = list(row['Pokemon'])

In [224]:
abil_poke = abil_poke.explode('Pokemon')
abil_poke.head()

,Name,Pokemon
0,Adaptability,"{'is_hidden': False, 'pokemon': {'name': 'eeve..."
0,Adaptability,"{'is_hidden': True, 'pokemon': {'name': 'corph..."
0,Adaptability,"{'is_hidden': True, 'pokemon': {'name': 'crawd..."
0,Adaptability,"{'is_hidden': True, 'pokemon': {'name': 'feeba..."
0,Adaptability,"{'is_hidden': False, 'pokemon': {'name': 'pory..."


In [225]:
abil_poke.reset_index(inplace=True)

In [226]:
ponte_abil_poke = pd.concat([abil_poke['Pokemon'].apply(pd.Series), abil_poke.drop('Pokemon', axis = 1)], axis = 1)

In [227]:
ponte_abil_poke = pd.concat([ponte_abil_poke['pokemon'].apply(pd.Series), ponte_abil_poke.drop('pokemon', axis = 1)], axis = 1)

In [228]:
ponte_abil_poke.head()

,name,url,is_hidden,slot,index,Name
0,eevee,https://pokeapi.co/api/v2/pokemon/133/,False,2,0,Adaptability
1,corphish,https://pokeapi.co/api/v2/pokemon/341/,True,3,0,Adaptability
2,crawdaunt,https://pokeapi.co/api/v2/pokemon/342/,True,3,0,Adaptability
3,feebas,https://pokeapi.co/api/v2/pokemon/349/,True,3,0,Adaptability
4,porygon-z,https://pokeapi.co/api/v2/pokemon/474/,False,1,0,Adaptability


In [229]:
ponte_abil_poke = ponte_abil_poke[['Name', 'name', 'is_hidden']]
ponte_abil_poke.columns = ['Ability', 'Pokemon', 'Hidden']
ponte_abil_poke.head()

,Ability,Pokemon,Hidden
0,Adaptability,eevee,False
1,Adaptability,corphish,True
2,Adaptability,crawdaunt,True
3,Adaptability,feebas,True
4,Adaptability,porygon-z,False


In [230]:
#Salvataggio sia in .csv che in .json
ponte_abil_poke.to_json("Tabelle ponte/ponte_abil_poke.json.gz", orient='records', lines=True, compression='gzip')
ponte_abil_poke.to_csv("Tabelle ponte/ponte_abil_poke.csv")

# PONTE MOSSE - POKEMON

In [231]:
move_poke = moves[["Name", "Learned_by"]]
move_poke.head()

,Name,Learned_by
0,Absorb,"[{'name': 'zubat', 'url': 'https://pokeapi.co/..."
1,Accelerock,"[{'name': 'lycanroc-midday', 'url': 'https://p..."
2,Acid,"[{'name': 'ekans', 'url': 'https://pokeapi.co/..."
3,Acid Armor,"[{'name': 'tentacool', 'url': 'https://pokeapi..."
4,Acid Spray,"[{'name': 'ekans', 'url': 'https://pokeapi.co/..."


In [232]:
for index, row in move_poke.iterrows():
    row['Learned_by'] = list(row['Learned_by'])

In [233]:
move_poke = move_poke.explode('Learned_by')
move_poke.head()

,Name,Learned_by
0,Absorb,"{'name': 'zubat', 'url': 'https://pokeapi.co/a..."
0,Absorb,"{'name': 'golbat', 'url': 'https://pokeapi.co/..."
0,Absorb,"{'name': 'oddish', 'url': 'https://pokeapi.co/..."
0,Absorb,"{'name': 'gloom', 'url': 'https://pokeapi.co/a..."
0,Absorb,"{'name': 'vileplume', 'url': 'https://pokeapi...."


In [234]:
move_poke.reset_index(inplace=True)

In [235]:
ponte_move_poke = pd.concat([move_poke['Learned_by'].apply(pd.Series), move_poke.drop('Learned_by', axis = 1)], axis = 1)

In [236]:
ponte_move_poke.head()

,0,name,url,index,Name
0,NaN,zubat,https://pokeapi.co/api/v2/pokemon/41/,0,Absorb
1,NaN,golbat,https://pokeapi.co/api/v2/pokemon/42/,0,Absorb
2,NaN,oddish,https://pokeapi.co/api/v2/pokemon/43/,0,Absorb
3,NaN,gloom,https://pokeapi.co/api/v2/pokemon/44/,0,Absorb
4,NaN,vileplume,https://pokeapi.co/api/v2/pokemon/45/,0,Absorb


In [237]:
ponte_move_poke = ponte_move_poke[['Name', 'name']]
ponte_move_poke.columns = ['Move', 'Pokemon']
ponte_move_poke.head()

,Move,Pokemon
0,Absorb,zubat
1,Absorb,golbat
2,Absorb,oddish
3,Absorb,gloom
4,Absorb,vileplume


In [238]:
#Salvataggio sia in .csv che in .json
ponte_move_poke.to_json("Tabelle ponte/ponte_move_poke.json.gz", orient='records', lines=True, compression='gzip')
ponte_move_poke.to_csv("Tabelle ponte/ponte_move_poke.csv")

# CORREZIONE INFORMAZIONI VARIANTI

In [ ]:
for index, row in pokemon.iterrows():
    if row['Variante'] == 'Mega':
        row['Generazione'] = 6
        row['Evolves_from'] = row['Species']
    if row['Variante'] == 'Alolan':
        row['Generazione'] = 7
    if row['Variante'] == 'Galarian':
        row['Generazione'] = 8 
    if row['Variante'] == 'Primal': #ArcheoGroudon e ArcheoKyogre
        row['Generazione'] = 6